# Set up the environment

Start by importing packages.

In [ ]:
# for gerrychain
import random
import math
import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from functools import partial
import pandas as pd
import geopandas as gpd
import pickle

from tqdm import tqdm

# for fancy map plotting
# import folium
# import mapclassify

Read in the PA VTDs and other info.

In [ ]:
# read in the shapefile as a dataframe
pa_vtds = gpd.read_file("./data/PA/PA.shp")

# see what's in the shapefile 
pa_vtds.head()

In [ ]:
# see the column names
pa_vtds["T16PRESR"]

In [ ]:
# convert the dataframe to a gerrychain-viable object
graph = Graph.from_geodataframe(pa_vtds)

In [ ]:
# fix the issues in the PA shapefile w.r.t. CD_2011 (see: https://github.com/mggg-states/PA-shapefiles )
graph.add_edge(7648,7635)
graph.add_edge(1247,1160)

# Set up the chain

Start by specifying election info (columns), population info, and the initial partition.

In [ ]:
# this is more-or-less following the tutorial here: https://gerrychain.readthedocs.io/en/latest/user/recom.html
elections = [
    Election("SEN10", {"Democratic": "SEN10D", "Republican": "SEN10R"}),
    Election("SEN12", {"Democratic": "USS12D", "Republican": "USS12R"}),
    Election("SEN16", {"Democratic": "T16SEND", "Republican": "T16SENR"}),
    Election("PRES12", {"Democratic": "PRES12D", "Republican": "PRES12R"}),
    Election("PRES16", {"Democratic": "T16PRESD", "Republican": "T16PRESR"})
]

# Population updater, for computing how close to equality the district
# populations are. "TOTPOP" is the population column from our shapefile.
my_updaters = {"population": updaters.Tally("TOTPOP", alias="population")}
# main difference here, name for the population column is different

# Election updaters, for computing election results using the vote totals
# from our shapefile.
election_updaters = {election.name: election for election in elections}
my_updaters.update(election_updaters)

initial_partition = GeographicPartition(graph, assignment="CD_2011", updaters=my_updaters)

Specify the ideal population, and set up the proposal step.

In [ ]:
# The ReCom proposal needs to know the ideal population for the districts so that
# we can improve speed by bailing early on unbalanced partitions.

ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)

# We use functools.partial to bind the extra parameters (pop_col, pop_target, epsilon, node_repeats)
# of the recom proposal.
proposal = partial(recom,
                   pop_col="TOTPOP",
                   pop_target=ideal_population,
                   epsilon=0.02,
                   node_repeats=2
                  )

Define the compactness constraint/bound, as well as the population constraint.

In [ ]:
compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    2*len(initial_partition["cut_edges"])
)

pop_constraint = constraints.within_percent_of_ideal_population(initial_partition, 0.02)

# to add a constraint that would bias a plan, probably add it here
# see also https://gerrychain.readthedocs.io/en/latest/api.html#module-gerrychain.constraints


In [ ]:
election_name = 'PRES16' 
bias_for_second_party = False

def increase_bias(partition):
    # accept anything if this is the first map
    if partition.parent is None:
        return True
    
    # get ElectionResults for current and previous map
    prev_partition = partition.parent

    current_elec_result = partition.updaters[election_name](partition)
    previous_elec_result = prev_partition.updaters[election_name](prev_partition)
    
    ## change here if we are interested in different "bias" metric
    delta = current_elec_result.efficiency_gap() - previous_elec_result.efficiency_gap()
    
    if bias_for_second_party:
        delta = -delta
        
    # accept immediately if "bias" is not decreasing
    if delta >= 0:
        return True
    
    # only get to this part of the code if delta < 0
    # in this case, still accept with some probability
    return random.random() <= math.exp(delta)


Piece everything together into a chain (which then gets run, eventually).

In [ ]:
num_steps = 300 # a shorter chain

chain = MarkovChain(
    proposal=proposal,
    constraints=[
        pop_constraint,
        compactness_bound #,bias_constraint
    ],
    #accept=accept.always_accept,
    accept = increase_bias,
    initial_state=initial_partition,
    total_steps=num_steps
)

# Run the chain

With everything set up, now run the chain.

In [ ]:
all_partitions = [] # keep track of the actual partitions
all_percents = [] # keep track of the democratic percentages
partisan_metric_values = [] # keep track of the partisan metrics
all_assignments = [] # something pickle-able

for partition in chain.with_progress_bar():
    all_partitions.append(partition)
    all_assignments.append(partition.assignment)
    partisan_metric_values.append(partition.updaters[election_name](partition).efficiency_gap())
    all_percents.append(sorted(partition["SEN12"].percents("Democratic")))


with open('chains/biased-for-first-party.pkl', 'wb') as f:
    pickle.dump(all_assignments, f)


In [ ]:
# with open('chains/biased-for-first-party.pkl', 'rb') as f:
#     test = pickle.load(f)

In [ ]:
plt.plot(partisan_metric_values)

Now produce some initial visualizations summarizing what's in the chain.

In [ ]:
data = pd.DataFrame(all_percents) #convert to a pd dataframe

# add the partitions to the dataframe as columns
for idx,p in enumerate(all_partitions):
    # read out the dictionary values of the partition
    pa_vtds[f"plan_{idx}"] = [p.assignment[ii] for ii in range(len(graph))]

fig, ax = plt.subplots(figsize=(8, 6))

# Draw 50% line
ax.axhline(0.5, color="#cccccc")

# Draw boxplot
data.boxplot(ax=ax, positions=range(len(data.columns)))

# Draw initial plan's Democratic vote %s (.iloc[0] gives the first row)
plt.plot(data.iloc[0], "ro")

# Annotate
ax.set_title("Comparing the 2011 plan to an ensemble")
ax.set_ylabel("Democratic vote % (Senate 2012)")
ax.set_xlabel("Sorted districts")
ax.set_ylim(0, 1)
ax.set_yticks([0, 0.25, 0.5, 0.75, 1])

plt.show()

# Visualize results

Folio can be tough to start with, but produces nice visuals with explorability. This portion is adapted from the tutoral here: https://kodu.ut.ee/~kmoch/geopython2019/L6/interactive-map-folium.html

See also examples and other ideas here: https://python-visualization.github.io/folium/quickstart.html

In [ ]:
# #this sets up a new map with prescribed view location and zoom, etc.
# m = folium.Map(location=[40, -77], zoom_start=6.4, control_scale=True, prefer_canvas=True, width=600, height=450)

# #this forces the map to be displayed
# m

In [ ]:
# # generate a new map and then draw the shapefiles
# m = folium.Map(location=[40, -77], zoom_start=6.4, control_scale=True, prefer_canvas=True, width=600, height=450)

# # Folium wants data as JSON objects
# # here, specify the columns to use in isolating the data
# pa_vtds["geoid"] = pa_vtds.index.astype(str)
# pa_plot_data = pa_vtds[["geoid","CD_2011","T16PRESD","T16PRESR","geometry"]]

# pa_jsontxt = pa_plot_data.to_json()
# pa_plot_data['geometry'] = pa_plot_data['geometry'].to_crs(epsg=4326) # make sure it's in the right projection

# #set up a "chloropleth" map 
# cp = folium.Choropleth(geo_data=pa_jsontxt,data=pa_plot_data,
#                   columns=['geoid', 'CD_2011'],
#                   key_on="feature.id",
#                   fill_opacity=0.5,
#                   line_opacity=0.2,
#                   line_color='white',
#                   line_weight=0,
#                   legend_name='District',
#                   name='CD_2011',
#                   highlight=True,
#                   fill_color='RdBu'
#                   ).add_to(m)

# # tooltip set-up using: https://stackoverflow.com/questions/70471888/text-as-tooltip-popup-or-labels-in-folium-choropleth-geojson-polygons

# # creating a state indexed version of the dataframe so we can lookup values

# # looping thru the geojson object and adding a new property(unemployment)
# # and assigning a value from our dataframe
# for s in cp.geojson.data['features']:
#     s['properties']['PRE16Dem'] = pa_vtds.loc[int(s['id']), 'T16PRESD']
#     s['properties']['PRE16Rep'] = pa_vtds.loc[int(s['id']), 'T16PRESR']

# # and finally adding a tooltip/hover to the choropleth's geojson
# folium.GeoJsonTooltip(['PRE16Dem', 'PRE16Rep']).add_to(cp.geojson)

# folium.LayerControl().add_to(m)
  

# m